# Benchmark of the Uploaded Program

## Import Necessary Libraries

In [115]:
import pandas as pd
import numpy as np
from obspy import UTCDateTime
import re
import datetime
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

## Input Data

In [116]:
tested_data = pd.read_csv("output.csv") # Output of the detection algorithm, to be tested
verification1 = pd.read_csv("Massachusetts/Hanscom TS.csv") # First airport close to the station
verification2 = pd.read_csv("Massachusetts/Fitchburg TS.csv") # Second airport close to the station
time_diff = -4

As the airports are usually $15-25$ km away from the site of the meteorological data, there are big uncertainties:
The thunderstorm can span across the airport but not about the seismic station or vice versa.
The storm can pass one site and get to the other site much later etc.

To minimize the resulting uncertainty, it is advisable to use a second airport in the other direction from the seismic station.
This gives a broader range for storms that may be pickup up but is still far from perfect.

## Processing

The data from *output.csv* needs to be converted into usable UTCDateTime objects.

In [117]:
def parse_time_interval(interval_str):
    """
    Parse a time interval string like "2016-04-08 16:30 – 2016-08-07 08:10"
    Returns tuple of (start_time, end_time) as UTCDateTime objects
    """
    # Check if the value is NaN or None
    if pd.isna(interval_str) or interval_str is None:
        return None, None
    
    # Convert to string and check if empty
    interval_str = str(interval_str).strip()
    if interval_str == '' or interval_str == 'nan':
        return None, None
    
    # Use a more specific pattern that looks for the dash between complete datetime strings
    # Pattern: YYYY-MM-DD HH:MM [dash] YYYY-MM-DD HH:MM
    datetime_pattern = r'(\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2})\s*[–—−-]\s*(\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2})'
    
    match = re.match(datetime_pattern, interval_str)
    
    if match:
        start_str = match.group(1).strip()
        end_str = match.group(2).strip()
    else:
        # Fallback: try to find two datetime patterns and assume the dash is between them
        datetime_matches = re.findall(r'\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}', interval_str)
        if len(datetime_matches) == 2:
            start_str = datetime_matches[0]
            end_str = datetime_matches[1]
        else:
            return None, None
    
    try:
        # Parse the datetime strings
        start_time = UTCDateTime(start_str)
        end_time = UTCDateTime(end_str)
        
        return start_time, end_time
        
    except Exception as e:
        print(f"Error parsing '{interval_str}': {e}")
        return None, None

The time stamps in the verification files needs to be converted to UTCDateTime too:

In [118]:
def get_time1(i):
    date_time = pd.to_datetime(verification1.loc[i, "valid"], format="%Y-%m-%d %H:%M")
    year = date_time.year
    month = date_time.month
    day = date_time.day
    hour = date_time.hour
    minute = date_time.minute
    time= UTCDateTime(year, month, day, hour, minute)
    time += timedelta(hours=time_diff)  # Adjust for time zone
    return time

def get_time2(i):
    date_time = pd.to_datetime(verification2.loc[i, "valid"], format="%Y-%m-%d %H:%M")
    year = date_time.year
    month = date_time.month
    day = date_time.day
    hour = date_time.hour
    minute = date_time.minute
    time= UTCDateTime(year, month, day, hour, minute)
    time += timedelta(hours=time_diff)  # Adjust for time zone
    return time

verification1_times = np.zeros(len(verification1), dtype=UTCDateTime)
verification2_times = np.zeros(len(verification2), dtype=UTCDateTime)

for i in range(len(verification1)):
    verification1_times[i] = get_time1(i)

for i in range(len(verification2)):
    verification2_times[i] = get_time2(i)


## Pointing out Thunderstorms from the Verification Files

The *ts_days* lists just save every day that has the weather code for thunderstorms in its data.
*heavy_ts_days* collects multiple thunderstorm entries shortly after one another and saves them as one big thunderstorm interval.

This is done for both of the airport data sets and afterwards they are merged to have one big list with all the days/time intervals for storms

In [119]:
ts_days1 = []
for i in range(len(verification1)):
    if "TS" in str(verification1.loc[i, "wxcodes"]) and verification1_times[i].date not in ts_days1:
        day_start = UTCDateTime(verification1_times[i].year, verification1_times[i].month, verification1_times[i].day, 0, 0, 0)
        day_end = day_start + timedelta(days=1)
        ts_days1.append(verification1_times[i].date)

In [120]:
heavy_ts_days1 = []
heavy_ts_times1 = []
ts_detections1 = []
current_ts = UTCDateTime(0)
ts_now = False
heavy_ts_now = False
detect_counter = 0
start_time_current_storm = UTCDateTime(0)
for i in range(len(verification1_times)-1):
    if "TS" not in str(verification1.loc[i, "wxcodes"]):
        continue
    else:
        day_start = UTCDateTime(verification1_times[i].year, verification1_times[i].month, verification1_times[i].day, 0, 0, 0)
        day_end = day_start + timedelta(days=1)
        if ts_now == False:
            current_ts = verification1_times[i]
            start_time_current_storm = verification1_times[i]
            ts_now = True
            detect_counter = 1
        elif verification1_times[i] - current_ts > 1.5 * 3600:
            if heavy_ts_now:
                if start_time_current_storm.date not in heavy_ts_days1:
                    heavy_ts_days1.append(start_time_current_storm.date)
                heavy_ts_times1.append((start_time_current_storm, current_ts))
                ts_detections1.append(detect_counter / float(current_ts - start_time_current_storm) * 3600)
            current_ts = verification1_times[i]
            start_time_current_storm = verification1_times[i]
            ts_now = True
            heavy_ts_now = False
            detect_counter = 1
        elif detect_counter == 1:
            detect_counter += 1
            current_ts = verification1_times[i]
        elif detect_counter == 2:
            detect_counter += 1
            current_ts = verification1_times[i]
            heavy_ts_now = True
        else:
            current_ts = verification1_times[i]
            detect_counter += 1
if "TS" not in str(verification1.loc[len(verification1_times)-1, "wxcodes"]):
    if heavy_ts_now:
        heavy_ts_days1.append(start_time_current_storm.date)
        heavy_ts_times1.append((start_time_current_storm, current_ts))
        ts_detections1.append(detect_counter/float(current_ts - start_time_current_storm) * 3600)

elif "TS" in str(verification1.loc[len(verification1_times)-1, "wxcodes"]):
    if ts_now:
        if verification1_times[len(verification1_times)-1] - current_ts > 1.5 * 3600:
            if heavy_ts_now:
                heavy_ts_days1.append(start_time_current_storm.date)
                heavy_ts_times1.append((start_time_current_storm, current_ts))
                ts_detections1.append(detect_counter/float(current_ts - start_time_current_storm) * 3600)
        else:
            day_start = UTCDateTime(verification1_times[m-1].year, verification1_times[m-1].month, verification1_times[m-1].day, 0, 0, 0)
            day_end = day_start + timedelta(days=1)
            if heavy_ts_now:
                current_ts = verification1_times[n-1]
                heavy_ts_times1.append((start_time_current_storm, current_ts))
                heavy_ts_days1.append(start_time_current_storm.date)
                ts_detections1.append(detect_counter/(current_ts - start_time_current_storm) * 3600)
            elif detect_counter == 2:
                current_ts = verification1_times[n-1]
                heavy_ts_now = True
                heavy_ts_times1.append((start_time_current_storm, current_ts))
                heavy_ts_days1.append(start_time_current_storm.date)
                ts_detections1.append(detect_counter/(current_ts - start_time_current_storm) * 3600)


In [121]:
ts_days2 = []
for i in range(len(verification2)):
    if "TS" in str(verification2.loc[i, "wxcodes"]) and verification2_times[i].date not in ts_days2:
        day_start = UTCDateTime(verification2_times[i].year, verification2_times[i].month, verification2_times[i].day, 0, 0, 0)
        day_end = day_start + timedelta(days=1)
        ts_days2.append(verification2_times[i].date)

In [122]:
heavy_ts_days2 = []
heavy_ts_times2 = []
ts_detections2 = []
current_ts = UTCDateTime(0)
ts_now = False
heavy_ts_now = False
detect_counter = 0
start_time_current_storm = UTCDateTime(0)
for i in range(len(verification2)-1):
    if "TS" not in str(verification2.loc[i, "wxcodes"]):
        continue
    else:
        day_start = UTCDateTime(verification2_times[i].year, verification2_times[i].month, verification2_times[i].day, 0, 0, 0)
        day_end = day_start + timedelta(days=1)
        if ts_now == False:
            current_ts = verification2_times[i]
            start_time_current_storm = verification2_times[i]
            ts_now = True
            detect_counter = 1
        elif verification2_times[i] - current_ts > 1.5 * 3600:
            if heavy_ts_now:
                if start_time_current_storm.date not in heavy_ts_days2:
                    heavy_ts_days2.append(start_time_current_storm.date)
                heavy_ts_times2.append((start_time_current_storm, current_ts))
                ts_detections2.append(detect_counter / float(current_ts - start_time_current_storm) * 3600)
            current_ts = verification2_times[i]
            start_time_current_storm = verification2_times[i]
            ts_now = True
            heavy_ts_now = False
            detect_counter = 1
        elif detect_counter == 1:
            detect_counter += 1
            current_ts = verification2_times[i]
        elif detect_counter == 2:
            detect_counter += 1
            current_ts = verification2_times[i]
            heavy_ts_now = True
        else:
            current_ts = verification2_times[i]
            detect_counter += 1
if "TS" not in str(verification2.loc[len(verification2)-1, "wxcodes"]):
    if heavy_ts_now:
        heavy_ts_days2.append(start_time_current_storm.date)
        heavy_ts_times2.append((start_time_current_storm, current_ts))
        ts_detections2.append(detect_counter/float(current_ts - start_time_current_storm) * 3600)

elif "TS" in str(verification2.loc[len(verification2)-1, "wxcodes"]):
    if ts_now:
        if verification2_times[len(verification2)-1] - current_ts > 1.5 * 3600:
            if heavy_ts_now:
                heavy_ts_days2.append(start_time_current_storm.date)
                heavy_ts_times2.append((start_time_current_storm, current_ts))
                ts_detections2.append(detect_counter/float(current_ts - start_time_current_storm) * 3600)
        else:
            day_start = UTCDateTime(verification2_times[len(verification2)-1].year, verification2_times[len(verification2)-1].month, verification2_times[len(verification2)-1].day, 0, 0, 0)
            day_end = day_start + timedelta(days=1)
            if heavy_ts_now:
                current_ts = verification2_times[len(verification2)-1]
                heavy_ts_times2.append((start_time_current_storm, current_ts))
                heavy_ts_days2.append(start_time_current_storm.date)
                ts_detections2.append(detect_counter/(current_ts - start_time_current_storm) * 3600)
            elif detect_counter == 2:
                current_ts = verification2_times[len(verification2)-1]
                heavy_ts_now = True
                heavy_ts_times2.append((start_time_current_storm, current_ts))
                heavy_ts_days2.append(start_time_current_storm.date)
                ts_detections2.append(detect_counter/(current_ts - start_time_current_storm) * 3600)

In [130]:
# Merge the two ts databases:
ts_days = ts_days1.copy()
for i in range(len(ts_days2)):
    if ts_days2[i] not in ts_days:
        ts_days.append(ts_days2[i])
ts_days.sort()

heavy_ts_days = heavy_ts_days1.copy()
for i in range(len(heavy_ts_days2)):
    if heavy_ts_days2[i] not in heavy_ts_days:
        heavy_ts_days.append(heavy_ts_days2[i])
heavy_ts_days.sort()

#merged time intervals:
all_intervals = heavy_ts_times1 + heavy_ts_times2
all_intervals.sort(key=lambda x: x[0])  # Sort by start time

heavy_ts_times = [all_intervals[0]]

for current in all_intervals[1:]:
    last_merged = heavy_ts_times[-1]
    if current[0] - last_merged[1] < 3 * 3600:  # Time intervals close to each other
        heavy_ts_times[-1] = (last_merged[0], max(last_merged[1], current[1]))
    else:
        heavy_ts_times.append(current)

Note that the merging of the time intervals still merges time intervals that are three hours apart from each other. This list doesn't have the goal to accurately display the duration of a thunderstorm but rather to give a reasonable time interval in which the storm might pass the seismic station between the two airports.

## Result

The results are collected in the following way:

If the predicted thunderstorm interval from our detection algorithm overlaps with one of the thunderstorm intervals from the verification data drawn from the airports, the predicted storm is considered a true positive.
It is still counted as a true positive if the proposed time span contains the weather code for a thunderstorm in one of the data sets.

If, at some other time during the day, there is a thunderstorm at one of the airports but the timeframe doesn't quite match the thunderstorm proposed by our algorithm, it is counted as likely true positive. 
As the airports are some distance away from the meteorological station, the thunderstorm or at least the responsible weather front might move slowly.

If for a predicted thunderstorm, there is no indication of a thunderstorm during that day in either of the data sets, the storm is counted towards false positives.

In [131]:
tp_counter = 0
un_counter = 0
fp_counter = 0
index = 0
for i in range(tested_data.shape[0]-1):
    for j in range(tested_data.shape[1]-1):
        tp = False
        if parse_time_interval(tested_data.iat[i +1, j+1]) == (None, None):
            continue
        start, end = parse_time_interval(tested_data.iat[i +1, j+1])
        start += timedelta(hours=-1)
        end += timedelta(hours=1)
        for k in range(len(heavy_ts_times)):
            if heavy_ts_times[k][0] <= start <= heavy_ts_times[k][1] or heavy_ts_times[k][0] <= end <= heavy_ts_times[k][1]:
                tp = True
                break
            elif start <= heavy_ts_times[k][0] <= end or start <= heavy_ts_times[k][1] <= end:
                tp = True
                break
        if tp:
            tp_counter += 1
        else:
            if start.date in ts_days:
                for k in range(index, len(verification1)):
                    if "TS" in str(verification1.loc[k, "wxcodes"]) and start <= verification1_times[k] <= end:
                        tp = True
                        index = k
                        break
                for k in range(k, len(verification2)):
                    if "TS" in str(verification2.loc[k, "wxcodes"]) and start <= verification2_times[k] <= end:
                        tp = True
                        index = k
                        break
                if tp:
                    tp_counter += 1
                else:
                    un_counter += 1
            else:
                fp_counter += 1

print(f"True Positives: {tp_counter}")
print(f"Likely True Positives: {un_counter}")
print(f"Likely False Positives: {fp_counter}")


True Positives: 58
Likely True Positives: 18
Likely False Positives: 22
